# Reading excel sheets into python as a matrix then multiply by an aggregation matrix to sum by category

#### Note this is only a proof of concept data is totally made up

In [132]:
import os
import pandas as pd
import numpy as np
import copy
from openpyxl import load_workbook
    
cwd = os.getcwd()
cwd = os.chdir('C:\Users\jojoseph\Desktop\Python Models\Python\IO')

print (os.getcwd())

C:\Users\jojoseph\Desktop\Python Models\Python\IO


In [133]:
#opening the social account matrix excel files from IMPLAN -(89 sectors summarized from study area data/study area explorer )
new_MRIO= 'new_MRIO.xlsx'# open file with social account matrices
reg_soc_acc_mat=pd.ExcelFile(new_MRIO)
print (reg_soc_acc_mat.sheet_names)

#region3=reg_soc_acc_mat.parse('region2')

[u'region0', u'region1', u'region2']


### Load aggregated sector names

In [134]:
sector_names = np.loadtxt('aggregated_sector_names.txt', usecols=1, delimiter="\'", dtype=str)[...,None]

sector_names

array([['Agriculture'],
       ['Mining'],
       ['Construction'],
       ['Durable Goods MFG'],
       ['Nondurable Goods MFG'],
       ['Trade, Transportaiton & Utilities'],
       ['Information'],
       ['Finance, Insurance & Real Estate'],
       ['Professional & Business Services'],
       ['Education & health Services'],
       ['Leisure & Hospitality'],
       ['Other Services'],
       ['Government'],
       ['Other Nonproducing Sectors']],
      dtype='|S33')

### Load the aggregation matrix

In [135]:
aggregation_matrix = np.loadtxt('aggregation_matrix.txt')

print aggregation_matrix

[[ 1.  1.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]


### For this exercise we will extract the first data column (employment) from each sheet and multiply it by the aggregation matrix

##### Note there are 534 rows in the excel sheets, the is because there are 6 regions (6x89=534) so we will just take the first region in our experiment

In [140]:
#opening the required files using while loop
dict_of_df = {}
aggregated_dict = {}
for sheet_name in reg_soc_acc_mat.sheet_names:   
    df = reg_soc_acc_mat.parse(sheet_name)  # sheet name
    dict_of_df[sheet_name] = df.iloc[:89,:]

dict_of_df

{u'region0':     ModCom ModOrig/Region  Employment          Sales  Employee Compensation  \
 0        1             AR    3.195816  988136.006115           53162.967118   
 1        2             AR    0.552749   34680.151438             720.678750   
 2        3             AR    0.272037   34441.204820            1303.430912   
 3        4             AR    0.182050  199407.832405            8752.788933   
 4        5             AR    0.044688   16299.301133            1491.864490   
 5        6             AR    0.033988   19863.844000            1696.146097   
 6        7             AR    0.100004   59007.165050            4678.281550   
 7        8             AR    0.001149    1333.868630              79.726994   
 8        9             AR    0.000000       0.245049               0.000000   
 9       10             AR    0.007174    1671.844976             350.025875   
 10      11             AR    0.005055     823.975495             263.143781   
 11      12             AR  

### now multiply the employment by the aggregation matrix.

In [137]:
for i in dict_of_df:
    print i, dict_of_df[i]
    aggregated_dict[i] = np.matmul(aggregation_matrix, dict_of_df[i])

region2     employment
0     151260.4
1         79.5
2         94.2
3         15.6
4         27.8
5          3.2
6        208.0
7          0.0
8          0.0
9          0.0
10      1287.4
11      2117.6
12        12.2
13      1044.3
14       100.9
15         0.0
16        91.1
17         0.0
18        13.3
19       170.0
20      1507.1
21       259.1
22       214.7
23         0.0
24         0.0
25         0.0
26         0.0
27         0.0
28         0.0
29         0.0
..         ...
59       993.6
60       231.5
61       996.8
62      1115.5
63       438.7
64       376.1
65       108.3
66       189.6
67         0.0
68         0.0
69         0.0
70        37.0
71         0.0
72         0.0
73         0.0
74         0.0
75         0.0
76         0.0
77         0.0
78         0.0
79       159.4
80         0.0
81       213.5
82       652.8
83         0.0
84       149.5
85         0.0
86         0.0
87         0.0
88         0.0

[89 rows x 1 columns]
region1     employment
0     151260.4
1

### Build the output array.

#### This code can be streamlined

In [139]:
out={}
#aggregated_dict
for x in aggregated_dict:
    out[x] = np.hstack((sector_names,aggregated_dict[x]))

out

{u'region0': array([['Agriculture', '5.25618610063'],
        ['Mining', '0.201870693439'],
        ['Construction', '1.01420391953'],
        ['Durable Goods MFG', '1.12381590301'],
        ['Nondurable Goods MFG', '0.56540197689'],
        ['Trade, Transportaiton & Utilities', '2.09691054164'],
        ['Information', '0.113888022988'],
        ['Finance, Insurance & Real Estate', '1.27252945089'],
        ['Professional & Business Services', '2.86764564349'],
        ['Education & health Services', '0.795770894041'],
        ['Leisure & Hospitality', '1.02733290169'],
        ['Other Services', '0.531906182772'],
        ['Government', '2.08865523005'],
        ['Other Nonproducing Sectors', '0.0']],
       dtype='|S33'), u'region1': array([['Agriculture', '151435.1'],
        ['Mining', '4575.1'],
        ['Construction', '69.0'],
        ['Durable Goods MFG', '1183.8'],
        ['Nondurable Goods MFG', '2295.2'],
        ['Trade, Transportaiton & Utilities', '2335.5'],
        ['I